# 内容
```meeting_0107```と内容はほぼ同じ。

# したいこと
* 01/15...```meeting_0107```にて```y```の出力が```ori_y```と全くおなじになっている原因解明
* 01/19...```num_select = 3, MCMC_step = 100, MCMC_step_each = 5, Max_temper = 500```で固定し、詳細を調べる

# 結果
多分わかった。でももっと効率的な方法があるはず。応急処置済み。

# 出力
* 各温度におけるエネルギーの推移
* 各温度での、全部のステップを経た後のy
* 各温度における最後の実験で、エネルギーが0となったものだけ表示されるdataframe

In [1]:
#import
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random, math, warnings
from sklearn.feature_selection import SelectKBest, f_regression

In [2]:
#valiable
num_select = 3

MCMC_step = 100
MCMC_step_each = 5

Max_temper = 1000
T = [j for j in range(1, Max_temper+1)]
len_T = len(T)

b = [1/ l for l in T]

In [3]:
X = pd.read_table('training_markers.txt', header=None,  sep ='\t')# 説明変数（こっちの特徴量上位5つ取り出す）


data_yields = pd.read_table('training_yields.txt', header=None,  sep ='\t')# 目的変数
#print(data_yields)
ori_y = data_yields.T.iloc[0]
print(ori_y)

y= data_yields.T.iloc[0]

num_sample = X.shape[0]#サンプル数
num_feature = X.shape[1]#特徴量の数

0     1
1     0
2     0
3     1
4     1
5     0
6     0
7     1
8     0
9     1
10    0
11    0
12    1
13    0
14    0
15    0
16    1
17    1
18    1
19    1
Name: 0, dtype: int64


In [4]:
type(X) #pandas.core.frame.DataFrame
X.shape #(20, 20)

type(y) #pandas.core.series.Series
y.shape #(20,)

type(y[0])

numpy.int64

In [5]:
#特徴量の分散や同じ特徴量が存在するのかを確認

from sklearn.feature_selection import VarianceThreshold
sel = VarianceThreshold(threshold=0)
sel.fit(X)

# get_supportで保持するデータのみをTrue値、そうでないものはFalse値を返します
print(sum(sel.get_support())) #20、すなわち同じ特徴量は存在していないことがわかった

20


In [6]:
def selection(num_select, X, y):
    # num_select個の特徴量を選択
    #print(f_regression(X,y))
    #print(np.linalg.norm(y))
    selector = SelectKBest(score_func=f_regression, k=num_select) #SelectKBest #f_regression...線形回帰
    selector.fit(X, y)
    mask_each = selector.get_support() #各特徴量を選択したか否かのmaskを取得
    return mask_each

In [7]:
def each_energy(mask_each): #毎回のmaskとyが引数、合致度合いの計算を行う
    energy_trait_sum = 0
    for i in range(len(mask_each)):
        if mask_each[i]==mask_ori[i]:
            energy_trait_sum += 1
    return energy_trait_sum #もとのyのときのmask(これをmaskとしている)との合致度合い（大きい⇒よく合致している）

In [8]:
def y_renewal(s: int):
    return 1-s

In [9]:
def Montecarlo(T, num_select, MCMC_step_each, E_first, y):
    E = []
    E.append(E_first)
    
    for j in range(MCMC_step_each):
        #print(j)
        ori_yi = y[j]
        #print(y[j])
        y[j] = y_renewal(y[j])
        #print(y)
        mask_new = selection(num_select, X, y)
        #print(mask_new)
        E_now = selected_X_energy - each_energy(mask_new)
        #print(E_now)
        E_last = E[-1]
        r = math.exp(-(E_now/T-E_last/T)) #温度付き
        R = np.random.uniform(0.0, 1.0)
        if R > r :
            y[j] = ori_yi #元に戻す
        else:
            E.append(E_now)
    return E, y #エネルギーのリスト #最後のyだけ(毎回更新しているので)

In [10]:
E_lists = []
Y_lists = []

mask_ori = selection(num_select, X, ori_y)
print(mask_ori)
selected_X_energy = each_energy(mask_ori)
ori_y = data_yields.T.iloc[0]

[False False False False False  True False False False False False False
 False False False False False False  True  True]


In [11]:
for k in range(len_T):
    E, y = Montecarlo(T[k], num_select, MCMC_step_each, 0, ori_y) #0=each_energy(mask_ori, y)-each_energy(mask_ori, y)
    #print(y)
    E_lists.append(E) #はじめのE=0は入れていないぞ
    Y_lists.append(y)
    data_yields = pd.read_table('training_yields.txt', header=None,  sep ='\t')# 目的変数
    ori_y = data_yields.T.iloc[0]
    #y = data_yields.T.iloc[0] #yの初期化
    #print(Y_lists[-1])
    print(Y_lists[-1]==y)

0     True
1     True
2     True
3     True
4     True
5     True
6     True
7     True
8     True
9     True
10    True
11    True
12    True
13    True
14    True
15    True
16    True
17    True
18    True
19    True
Name: 0, dtype: bool
0     True
1     True
2     True
3     True
4     True
5     True
6     True
7     True
8     True
9     True
10    True
11    True
12    True
13    True
14    True
15    True
16    True
17    True
18    True
19    True
Name: 0, dtype: bool
0     True
1     True
2     True
3     True
4     True
5     True
6     True
7     True
8     True
9     True
10    True
11    True
12    True
13    True
14    True
15    True
16    True
17    True
18    True
19    True
Name: 0, dtype: bool
0     True
1     True
2     True
3     True
4     True
5     True
6     True
7     True
8     True
9     True
10    True
11    True
12    True
13    True
14    True
15    True
16    True
17    True
18    True
19    True
Name: 0, dtype: bool
0     True
1     True
2     True
3  

0     True
1     True
2     True
3     True
4     True
5     True
6     True
7     True
8     True
9     True
10    True
11    True
12    True
13    True
14    True
15    True
16    True
17    True
18    True
19    True
Name: 0, dtype: bool
0     True
1     True
2     True
3     True
4     True
5     True
6     True
7     True
8     True
9     True
10    True
11    True
12    True
13    True
14    True
15    True
16    True
17    True
18    True
19    True
Name: 0, dtype: bool
0     True
1     True
2     True
3     True
4     True
5     True
6     True
7     True
8     True
9     True
10    True
11    True
12    True
13    True
14    True
15    True
16    True
17    True
18    True
19    True
Name: 0, dtype: bool
0     True
1     True
2     True
3     True
4     True
5     True
6     True
7     True
8     True
9     True
10    True
11    True
12    True
13    True
14    True
15    True
16    True
17    True
18    True
19    True
Name: 0, dtype: bool
0     True
1     True
2     True
3  

Name: 0, dtype: bool
0     True
1     True
2     True
3     True
4     True
5     True
6     True
7     True
8     True
9     True
10    True
11    True
12    True
13    True
14    True
15    True
16    True
17    True
18    True
19    True
Name: 0, dtype: bool
0     True
1     True
2     True
3     True
4     True
5     True
6     True
7     True
8     True
9     True
10    True
11    True
12    True
13    True
14    True
15    True
16    True
17    True
18    True
19    True
Name: 0, dtype: bool
0     True
1     True
2     True
3     True
4     True
5     True
6     True
7     True
8     True
9     True
10    True
11    True
12    True
13    True
14    True
15    True
16    True
17    True
18    True
19    True
Name: 0, dtype: bool
0     True
1     True
2     True
3     True
4     True
5     True
6     True
7     True
8     True
9     True
10    True
11    True
12    True
13    True
14    True
15    True
16    True
17    True
18    True
19    True
Name: 0, dtype: bool
0     True
1   

0     True
1     True
2     True
3     True
4     True
5     True
6     True
7     True
8     True
9     True
10    True
11    True
12    True
13    True
14    True
15    True
16    True
17    True
18    True
19    True
Name: 0, dtype: bool
0     True
1     True
2     True
3     True
4     True
5     True
6     True
7     True
8     True
9     True
10    True
11    True
12    True
13    True
14    True
15    True
16    True
17    True
18    True
19    True
Name: 0, dtype: bool
0     True
1     True
2     True
3     True
4     True
5     True
6     True
7     True
8     True
9     True
10    True
11    True
12    True
13    True
14    True
15    True
16    True
17    True
18    True
19    True
Name: 0, dtype: bool
0     True
1     True
2     True
3     True
4     True
5     True
6     True
7     True
8     True
9     True
10    True
11    True
12    True
13    True
14    True
15    True
16    True
17    True
18    True
19    True
Name: 0, dtype: bool
0     True
1     True
2     True
3  

0     True
1     True
2     True
3     True
4     True
5     True
6     True
7     True
8     True
9     True
10    True
11    True
12    True
13    True
14    True
15    True
16    True
17    True
18    True
19    True
Name: 0, dtype: bool
0     True
1     True
2     True
3     True
4     True
5     True
6     True
7     True
8     True
9     True
10    True
11    True
12    True
13    True
14    True
15    True
16    True
17    True
18    True
19    True
Name: 0, dtype: bool
0     True
1     True
2     True
3     True
4     True
5     True
6     True
7     True
8     True
9     True
10    True
11    True
12    True
13    True
14    True
15    True
16    True
17    True
18    True
19    True
Name: 0, dtype: bool
0     True
1     True
2     True
3     True
4     True
5     True
6     True
7     True
8     True
9     True
10    True
11    True
12    True
13    True
14    True
15    True
16    True
17    True
18    True
19    True
Name: 0, dtype: bool
0     True
1     True
2     True
3  

...

0     True
1     True
2     True
3     True
4     True
5     True
6     True
7     True
8     True
9     True
10    True
11    True
12    True
13    True
14    True
15    True
16    True
17    True
18    True
19    True
Name: 0, dtype: bool
0     True
1     True
2     True
3     True
4     True
5     True
6     True
7     True
8     True
9     True
10    True
11    True
12    True
13    True
14    True
15    True
16    True
17    True
18    True
19    True
Name: 0, dtype: bool
0     True
1     True
2     True
3     True
4     True
5     True
6     True
7     True
8     True
9     True
10    True
11    True
12    True
13    True
14    True
15    True
16    True
17    True
18    True
19    True
Name: 0, dtype: bool
0     True
1     True
2     True
3     True
4     True
5     True
6     True
7     True
8     True
9     True
10    True
11    True
12    True
13    True
14    True
15    True
16    True
17    True
18    True
19    True
Name: 0, dtype: bool
0     True
1     True
2     True
3  

0     True
1     True
2     True
3     True
4     True
5     True
6     True
7     True
8     True
9     True
10    True
11    True
12    True
13    True
14    True
15    True
16    True
17    True
18    True
19    True
Name: 0, dtype: bool
0     True
1     True
2     True
3     True
4     True
5     True
6     True
7     True
8     True
9     True
10    True
11    True
12    True
13    True
14    True
15    True
16    True
17    True
18    True
19    True
Name: 0, dtype: bool
0     True
1     True
2     True
3     True
4     True
5     True
6     True
7     True
8     True
9     True
10    True
11    True
12    True
13    True
14    True
15    True
16    True
17    True
18    True
19    True
Name: 0, dtype: bool
0     True
1     True
2     True
3     True
4     True
5     True
6     True
7     True
8     True
9     True
10    True
11    True
12    True
13    True
14    True
15    True
16    True
17    True
18    True
19    True
Name: 0, dtype: bool
0     True
1     True
2     True
3  

0     True
1     True
2     True
3     True
4     True
5     True
6     True
7     True
8     True
9     True
10    True
11    True
12    True
13    True
14    True
15    True
16    True
17    True
18    True
19    True
Name: 0, dtype: bool
0     True
1     True
2     True
3     True
4     True
5     True
6     True
7     True
8     True
9     True
10    True
11    True
12    True
13    True
14    True
15    True
16    True
17    True
18    True
19    True
Name: 0, dtype: bool
0     True
1     True
2     True
3     True
4     True
5     True
6     True
7     True
8     True
9     True
10    True
11    True
12    True
13    True
14    True
15    True
16    True
17    True
18    True
19    True
Name: 0, dtype: bool
0     True
1     True
2     True
3     True
4     True
5     True
6     True
7     True
8     True
9     True
10    True
11    True
12    True
13    True
14    True
15    True
16    True
17    True
18    True
19    True
Name: 0, dtype: bool
0     True
1     True
2     True
3  

Name: 0, dtype: bool
0     True
1     True
2     True
3     True
4     True
5     True
6     True
7     True
8     True
9     True
10    True
11    True
12    True
13    True
14    True
15    True
16    True
17    True
18    True
19    True
Name: 0, dtype: bool
0     True
1     True
2     True
3     True
4     True
5     True
6     True
7     True
8     True
9     True
10    True
11    True
12    True
13    True
14    True
15    True
16    True
17    True
18    True
19    True
Name: 0, dtype: bool
0     True
1     True
2     True
3     True
4     True
5     True
6     True
7     True
8     True
9     True
10    True
11    True
12    True
13    True
14    True
15    True
16    True
17    True
18    True
19    True
Name: 0, dtype: bool
0     True
1     True
2     True
3     True
4     True
5     True
6     True
7     True
8     True
9     True
10    True
11    True
12    True
13    True
14    True
15    True
16    True
17    True
18    True
19    True
Name: 0, dtype: bool
0     True
1   

0     True
1     True
2     True
3     True
4     True
5     True
6     True
7     True
8     True
9     True
10    True
11    True
12    True
13    True
14    True
15    True
16    True
17    True
18    True
19    True
Name: 0, dtype: bool
0     True
1     True
2     True
3     True
4     True
5     True
6     True
7     True
8     True
9     True
10    True
11    True
12    True
13    True
14    True
15    True
16    True
17    True
18    True
19    True
Name: 0, dtype: bool
0     True
1     True
2     True
3     True
4     True
5     True
6     True
7     True
8     True
9     True
10    True
11    True
12    True
13    True
14    True
15    True
16    True
17    True
18    True
19    True
Name: 0, dtype: bool
0     True
1     True
2     True
3     True
4     True
5     True
6     True
7     True
8     True
9     True
10    True
11    True
12    True
13    True
14    True
15    True
16    True
17    True
18    True
19    True
Name: 0, dtype: bool
0     True
1     True
2     True
3  

In [12]:
Y_lists[0]==ori_y

0     False
1      True
2      True
3      True
4     False
5      True
6      True
7      True
8      True
9      True
10     True
11     True
12     True
13     True
14     True
15     True
16     True
17     True
18     True
19     True
Name: 0, dtype: bool

In [13]:
Y_lists[1]

0     0
1     1
2     1
3     1
4     1
5     0
6     0
7     1
8     0
9     1
10    0
11    0
12    1
13    0
14    0
15    0
16    1
17    1
18    1
19    1
Name: 0, dtype: int64

In [14]:
Y_lists_df_middle = pd.DataFrame()
Y_lists_df_middle.index.name = 'y'
Y_lists_df_middle.columns.name = 'temperature'
for t in range(len_T):
    Y_lists_df_middle[t+1]=Y_lists[t]
data_yields = pd.read_table('training_yields.txt', header=None,  sep ='\t')# 目的変数
ori_y = data_yields.T.iloc[0]
Y_lists_df_middle['ori']=ori_y
Y_lists_df_middle

temperature,1,2,3,4,5,6,7,8,9,10,...,492,493,494,495,496,497,498,499,500,ori
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,0,1,0,1,1,0,0,1,0,1,...,1,1,1,1,1,1,1,1,1,0
2,0,1,1,1,1,1,0,1,0,1,...,1,1,1,1,1,1,1,1,1,0
3,1,1,1,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,0,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1


In [15]:
E_lists_df = pd.DataFrame(index=[], columns=[])

#del E_lists[0]#元の条件削除⇒len(E_lists)=MCMC_step_each となるはず

i = random.randrange(len_T-1)
#r = math.exp(-E_lists[i+1][-1]*b[i+1]-(-E_lists[i][-1]*b[i])) #温度付き
r = math.exp((b[i+1]-b[i])* (E_lists[i+1][-1]-E_lists[i][-1]))
rnd = np.random.uniform(0.0, 1.0)
if rnd < r:
    E_lists[i][-1], E_lists[i+1][-1] = E_lists[i+1][-1], E_lists[i][-1]

In [16]:
Y_lists_df_middle = pd.DataFrame()
Y_lists_df_middle.index.name = 'y'
Y_lists_df_middle.columns.name = 'temperature'
for t in range(len_T):
    Y_lists_df_middle[t+1]=Y_lists[t]
data_yields = pd.read_table('training_yields.txt', header=None,  sep ='\t')# 目的変数
ori_y = data_yields.T.iloc[0]
Y_lists_df_middle['ori']=ori_y
Y_lists_df_middle

temperature,1,2,3,4,5,6,7,8,9,10,...,492,493,494,495,496,497,498,499,500,ori
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,0,1,0,1,1,0,0,1,0,1,...,1,1,1,1,1,1,1,1,1,0
2,0,1,1,1,1,1,0,1,0,1,...,1,1,1,1,1,1,1,1,1,0
3,1,1,1,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,0,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1


In [17]:
data_yields.T.iloc[0]

0     1
1     0
2     0
3     1
4     1
5     0
6     0
7     1
8     0
9     1
10    0
11    0
12    1
13    0
14    0
15    0
16    1
17    1
18    1
19    1
Name: 0, dtype: int64

In [18]:
# 各温度におけるエネルギーの推移
for j in range(int(MCMC_step/MCMC_step_each)):

    for k in range(len_T):
        E, y = Montecarlo(T[k], num_select, MCMC_step_each, E_lists[k][-1], Y_lists[k])
        E_lists.append(E)
        Y_lists.append(y)

    E_lists_df[j] = E_lists[0:len_T]
    del E_lists[0:len_T]#1つ前のループで作られたリストの削除
    del Y_lists[0:len_T]

    i = random.randrange(len_T-1)
    #r = math.exp(-E_lists[i+1][-1]*b[i+1]-(-E_lists[i][-1]*b[i])) #温度付き
    r = math.exp((b[i+1]-b[i])* (E_lists[i+1][-1]-E_lists[i][-1]))
    rnd = np.random.uniform(0.0, 1.0)
    if rnd < r:
        E_lists[i][-1], E_lists[i+1][-1] = E_lists[i+1][-1], E_lists[i][-1]


#各温度でMCMC_step回動作させたことで、{MCMC_step_each個のエネルギーと最終y}のデータがkセットできた
#最後に知りたいのは、最終ykセットと、エネルギーの動き
E_lists_df.index.name = 'temperature'
E_lists_df.columns.name = 'each MCMC step'
E_lists_df

each MCMC step,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
temperature,,,,,,,,,,,,,,,,,,,,
0,"[0, 0, 2]","[2, 2, 4, 4]","[4, 2, 0, 2]","[2, 2, 4, 4, 4]","[4, 4, 2, 0, 2]","[2, 2, 4, 4, 4]","[4, 4, 4, 4, 2]","[2, 2, 0]","[0, 0, 2, 2]","[2, 2, 0]","[0, 0]","[0, 0]","[0, 0]","[0, 0]","[0, 0, 2]","[2, 2, 2, 2]","[2, 2, 0]","[0, 0]","[0, 0, 2, 4, 4]","[4, 4, 2, 2, 0]"
1,"[0, 0, 2, 2]","[2, 2, 0]","[0, 0, 2, 4, 4, 4]","[4, 4, 4, 2, 2, 0]","[0, 0, 2, 4, 4, 4]","[4, 4, 4, 2, 2, 0]","[0, 0, 2, 4, 4, 4]","[4, 4, 4, 2, 2, 0]","[0, 0, 2]","[2, 2, 0, 2, 2]","[2, 4, 4, 4, 2]","[2, 4, 2, 0]","[0, 0, 2]","[2, 2, 0, 2]","[2, 2, 4, 4, 4]","[4, 4, 4, 4, 2]","[2, 2, 0, 2]","[2, 2, 0]","[0, 0]","[0, 0, 2, 4, 4]"
2,"[0, 0, 2]","[2, 2, 4, 2, 4, 4]","[4, 4, 2, 4, 4, 2]","[2, 2, 4, 2, 4, 4]","[4, 4, 2, 4, 4, 2]","[2, 2, 0, 2]","[2, 2, 4, 4, 4, 4]","[4, 4, 4, 4, 0, 2]","[2, 2, 4, 4, 4, 4]","[4, 4, 4, 4, 0, 2]","[2, 2, 4, 4, 4, 4]","[4, 4, 4, 4, 0, 2]","[2, 2, 2, 2]","[2, 4, 4, 2, 4]","[4, 6, 4, 2, 4, 4]","[4, 2, 4, 4, 2, 4]","[4, 4, 2, 2, 2]","[2, 4, 4, 4, 4]","[4, 2, 2, 0]","[0, 0, 2, 4, 4, 4]"
3,"[0, 0, 2, 2, 4, 4]","[4, 4, 4, 4, 2, 0]","[0, 0, 2, 4]","[4, 4, 2, 4, 4]","[4, 2, 4, 4, 2, 4]","[4, 6, 4, 2, 4, 4]","[4, 2, 4, 4, 2]","[2, 4, 2, 0, 4, 4]","[4, 2, 4, 4, 4, 2]","[2, 4, 2, 0, 2]","[2, 2, 4, 4, 4, 4]","[4, 4, 4, 4, 0, 2]","[2, 2, 4, 4, 4]","[4, 4, 4, 4, 2]","[2, 2, 0, 2, 4]","[4, 4, 6, 4, 4, 4]","[4, 4, 2, 0, 2]","[2, 2, 4, 6, 4, 4]","[4, 4, 4, 2, 0, 2]","[2, 2, 4, 4]"
4,"[0, 0, 2, 2, 4, 4]","[4, 4, 4, 4, 2, 0]","[0, 0, 2, 2, 4, 4]","[4, 4, 4, 4, 2, 0]","[0, 0, 2, 4, 4]","[4, 4, 4, 4, 4, 2]","[2, 2, 0, 2, 2]","[2, 4, 4, 4, 2]","[2, 4, 2, 0, 2]","[2, 2, 4, 4, 4, 4]","[4, 4, 4, 4, 0, 2]","[2, 2, 4, 4, 4, 4]","[4, 4, 4, 4, 0, 2]","[2, 2, 4, 4, 4, 4]","[4, 4, 4, 4, 0]","[0, 0, 2, 2, 4, 4]","[4, 4, 4, 4, 2, 0]","[0, 0, 2, 4, 4]","[4, 4, 4, 4, 4, 2]","[2, 2, 0, 2, 2]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,"[0, 0, 2, 2, 4, 4]","[4, 4, 4, 4, 2, 0]","[0, 0, 2, 2, 4, 4]","[4, 4, 4, 4, 2, 0]","[0, 0, 2, 2, 4, 4]","[4, 4, 4, 4, 2, 0]","[0, 0, 2, 2, 4, 4]","[4, 4, 4, 4, 2, 0]","[0, 0, 2, 2, 4, 4]","[4, 4, 4, 4, 2, 0]","[0, 0, 2, 2, 4, 4]","[4, 4, 4, 4, 2, 0]","[0, 0, 2, 2, 4, 4]","[4, 4, 4, 4, 2, 0]","[0, 0, 2, 2, 4, 4]","[4, 4, 4, 4, 2, 0]","[0, 0, 2, 2, 4, 4]","[4, 4, 4, 4, 2, 0]","[0, 0, 2, 2, 4, 4]","[4, 4, 4, 4, 2, 0]"
496,"[0, 0, 2, 2, 4, 4]","[4, 4, 4, 4, 2, 0]","[0, 0, 2, 2, 4, 4]","[4, 4, 4, 4, 2, 0]","[0, 0, 2, 2, 4, 4]","[4, 4, 4, 4, 2, 0]","[0, 0, 2, 2, 4, 4]","[4, 4, 4, 4, 2, 0]","[0, 0, 2, 2, 4, 4]","[4, 4, 4, 4, 2, 0]","[0, 0, 2, 2, 4, 4]","[4, 4, 4, 4, 2, 0]","[0, 0, 2, 2, 4, 4]","[4, 4, 4, 4, 2, 0]","[0, 0, 2, 2, 4, 4]","[4, 4, 4, 4, 2, 0]","[0, 0, 2, 2, 4, 4]","[4, 4, 4, 4, 2, 0]","[0, 0, 2, 2, 4, 4]","[4, 4, 4, 4, 2, 0]"
497,"[0, 0, 2, 2, 4, 4]","[4, 4, 4, 4, 2, 0]","[0, 0, 2, 2, 4, 4]","[4, 4, 4, 4, 2, 0]","[0, 0, 2, 2, 4, 4]","[4, 4, 4, 4, 2, 0]","[0, 0, 2, 2, 4, 4]","[4, 4, 4, 4, 2, 0]","[0, 0, 2, 2, 4, 4]","[4, 4, 4, 4, 2, 0]","[0, 0, 2, 2, 4, 4]","[4, 4, 4, 4, 2, 0]","[0, 0, 2, 2, 4, 4]","[4, 4, 4, 4, 2, 0]","[0, 0, 2, 2, 4, 4]","[4, 4, 4, 4, 2, 0]","[0, 0, 2, 2, 4, 4]","[4, 4, 4, 4, 2, 0]","[0, 0, 2, 2, 4, 4]","[4, 4, 4, 4, 2, 0]"


In [19]:
E_lists_df[16][:] #列指定となってしまった

#以下の書き方だと行指定になった
print(E_lists_df.iloc[16])
print(E_lists_df.iloc[17])
E_lists_df.iloc[0]

each MCMC step
0     [0, 0, 2, 2, 4, 4]
1     [4, 4, 4, 4, 2, 0]
2     [0, 0, 2, 2, 4, 4]
3     [4, 4, 4, 4, 2, 0]
4     [0, 0, 2, 2, 4, 4]
5     [4, 4, 4, 4, 2, 0]
6     [0, 0, 2, 2, 4, 4]
7     [4, 4, 4, 4, 2, 0]
8     [0, 0, 2, 2, 4, 4]
9     [4, 4, 4, 4, 2, 0]
10    [0, 0, 2, 2, 4, 4]
11    [4, 4, 4, 4, 2, 0]
12       [0, 0, 2, 2, 4]
13       [4, 6, 4, 2, 0]
14    [0, 0, 2, 2, 4, 4]
15    [4, 4, 4, 4, 2, 0]
16       [0, 0, 2, 2, 4]
17    [4, 6, 4, 2, 4, 4]
18    [4, 2, 4, 4, 2, 4]
19       [4, 4, 2, 2, 2]
Name: 16, dtype: object
each MCMC step
0     [0, 0, 2, 2, 4, 4]
1     [4, 4, 4, 4, 2, 0]
2     [0, 0, 2, 2, 4, 4]
3     [4, 4, 4, 4, 2, 0]
4     [0, 0, 2, 2, 4, 4]
5     [4, 4, 4, 4, 2, 0]
6     [0, 0, 2, 2, 4, 4]
7     [4, 4, 4, 4, 2, 0]
8     [0, 0, 2, 2, 4, 4]
9     [4, 4, 4, 4, 2, 0]
10    [0, 0, 2, 2, 4, 4]
11    [4, 4, 4, 4, 2, 0]
12          [0, 0, 2, 4]
13    [4, 4, 6, 4, 4, 4]
14    [4, 4, 2, 4, 2, 4]
15    [4, 4, 6, 4, 4, 4]
16    [4, 4, 2, 4, 2, 4]
17    [4, 4, 6, 4, 4,

each MCMC step
0           [0, 0, 2]
1        [2, 2, 4, 4]
2        [4, 2, 0, 2]
3     [2, 2, 4, 4, 4]
4     [4, 4, 2, 0, 2]
5     [2, 2, 4, 4, 4]
6     [4, 4, 4, 4, 2]
7           [2, 2, 0]
8        [0, 0, 2, 2]
9           [2, 2, 0]
10             [0, 0]
11             [0, 0]
12             [0, 0]
13             [0, 0]
14          [0, 0, 2]
15       [2, 2, 2, 2]
16          [2, 2, 0]
17             [0, 0]
18    [0, 0, 2, 4, 4]
19    [4, 4, 2, 2, 0]
Name: 0, dtype: object

In [20]:
data_yields.T.iloc[0]

0     1
1     0
2     0
3     1
4     1
5     0
6     0
7     1
8     0
9     1
10    0
11    0
12    1
13    0
14    0
15    0
16    1
17    1
18    1
19    1
Name: 0, dtype: int64

In [21]:
# 各温度での、全部のステップを経た後のy
Y_lists_df = pd.DataFrame()
Y_lists_df.index.name = 'y'
Y_lists_df.columns.name = 'temperature'
for t in range(len_T):
    Y_lists_df[t+1]=Y_lists[t]
data_yields = pd.read_table('training_yields.txt', header=None,  sep ='\t')# 目的変数
ori_y = data_yields.T.iloc[0]
Y_lists_df['ori']=ori_y
Y_lists_df

temperature,1,2,3,4,5,6,7,8,9,10,...,492,493,494,495,496,497,498,499,500,ori
0,1,0,0,0,1,0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,1
1,0,1,0,1,1,1,0,1,1,1,...,1,1,1,1,1,1,1,1,1,0
2,0,0,0,1,1,0,1,1,0,0,...,1,1,1,1,1,1,1,1,1,0
3,1,1,1,1,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,1
4,0,1,1,1,1,0,1,0,1,1,...,0,0,0,0,0,0,0,0,0,1
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1


In [22]:
#E_lists[len_T - 1][-1] #IndexError: list index out of range
E_lists[0][-1] == 0 #False
E_lists[495][-1] == 0 #False
E_lists[16][-1] == 0 #True(t = 17)

none_count = 0
for t in range(len_T):
    if E_lists[t][-1] == None:
        none_count += 1

In [23]:
#各温度における最後の実験で、エネルギーが0となったものだけ表示されるようなdataframe

Y_lists_lastE0_df = pd.DataFrame()
for t in range(len_T):
    if E_lists[t][-1] == 0:
        Y_lists_lastE0_df[t+1]=Y_lists[t]
        
data_yields = pd.read_table('training_yields.txt', header=None,  sep ='\t')# 目的変数
ori_y = data_yields.T.iloc[0]

Y_lists_lastE0_df['ori']=ori_y

Y_lists_lastE0_df.index.name = 'y'
Y_lists_lastE0_df.columns.name = 'temperature'
Y_lists_lastE0_df

temperature,3,11,ori
y,,,
0,0,0,1
1,0,0,0
2,0,0,0
3,1,1,1
4,1,1,1
5,0,0,0
6,0,0,0
7,1,1,1
8,0,0,0


In [24]:
data_yields

,0
0,1
1,0
2,0
3,1
4,1
5,0
6,0
7,1
8,0
9,1


## 発生している問題

```ori_y```の値が変化する（しないように設定したつもりなのに）

### 確認したこと

- ```training_yields.txt```これは変わっていなかった
- ```data_yields```は変化している

### 行った処置
```ori_y```を使う際は、毎回

```data_yields = pd.read_table('training_yields.txt', header=None,  sep ='\t') #目的変数```

```ori_y = data_yields.T.iloc[0]```

を入れることにした。

### 処置が効果のある理由